<a href="https://colab.research.google.com/github/VaiSuliafu/CS6350_MachineLearning/blob/master/Pytorch_NN_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
# imports
import torch # entire library
import torch.nn as nn # nn modules and loss functions
import torch.optim as optim # optimization algorithms
import torch.nn.functional as F # functions without paramters (activation functions)
from torch.utils.data import DataLoader, Dataset # DataLoader gives us easier dataset management 
import torchvision.datasets as datasets # pytorch standard datasets
import torchvision.transforms as transforms # transformations we can perform on our dataset

import requests # for getting M4 testing/training dataset
import pandas as pd
import numpy as np

from fbprophet import Prophet

import datetime

from sklearn.preprocessing import MinMaxScaler

In [91]:
# This section holds a class for each of the models to be tested

# Create Fully Connected Network
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

# Create CNN 
class CNN(nn.Module):
  def __init__(self, in_channel = 1, num_classes = 10):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1)) # same convolution
    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1)) # same convolution 
    self.fc1 = nn.Linear(16*7*7, num_classes)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)

    return x

# Create an RNN
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

  def forward(self, x):
    # initial hidden state
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

    # Forward Prop
    out, _ = self.rnn(x, h0)
    out = out.reshape(out.shape[0], -1)
    out = self.fc(out)
    return out

# Create a GRU
class GRU(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(GRU, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

  def forward(self, x):
    # initial hidden state
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

    # Forward Prop
    out, _ = self.gru(x, h0)
    out = out.reshape(out.shape[0], -1)
    out = self.fc(out)
    return out

# Create an LSTM
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(LSTM, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

  def forward(self, x):
    # initial hidden state
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

    # Forward Prop
    out, _ = self.lstm(x, (h0, c0))
    out = out.reshape(out.shape[0], -1)
    out = self.fc(out)
    return out

# Bi-directional LSTM
class BRNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(BRNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
    self.fc = nn.Linear(hidden_size*2, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)

    out, _ = self.lstm(x, (h0, c0))
    out = self.fc(out[:, -1, :])

    return out

# Momentum LSTM Cell
class MomentumLSTMCell(nn.Module):
  def __init__(self, input_size, hidden_size, mu, epsilon, bias=True, fg_init=1.0):
    super(MomentumLSTMCell, self).__init__()
    self.input_size = input_size
    self.hidden_size =  hidden_size
    self.bias = bias
    self.fg_init = fg_init
    self.x2h = nn.Linear(input_size, 4 * hidden_size, bias=bias)
    self.h2h = nn.Linear(hidden_size, 4 * hidden_size, bias=bias)

    # for momentum net
    self.mu = mu
    self.epsilon = epsilon

    self.reset_parameters(hidden_size)

  def reset_parameters(self, hidden_size):
    nn.init.orthogonal_(self.x2h.weight)
    nn.init.eye_(self.h2h.weight)
    nn.init.zeros_(self.x2h.bias)
    self.x2h.bias.data[hidden_size:(2 * hidden_size)].fill_(self.fg_init)
    nn.init.zeros_(self.h2h.bias)
    self.h2h.bias.data[hidden_size:(2 * hidden_size)].fill_(self.fg_init)

  def forward(self, x, hidden, v):

    hx, cx = hidden
    x = x.squeeze()
    x = x.view(-1, x.size(1))
    v = v.view(-1, v.size(1))

    vy = self.mu * v + self.epsilon * self.x2h(x)
    
    gates = vy + self.h2h(hx).squeeze()
    gates = gates.squeeze()

    i,f,o,g = gates.chunk(4, 1)
    i = torch.sigmoid(i)
    f = torch.sigmoid(f)
    o = torch.sigmoid(o)
    g = torch.tanh(g)

    cy = (cx * f) + (i * g)
    hy = o * torch.tanh(cy)

    return hy, (hy, cy), vy

# Create a MomentumLSTM
class MomentumLSTM(nn.Module):
  def __init__(self, input_size, hidden_size, mu, epsilon, bias=True, fg_init=1.0):
    super(MomentumLSTM, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.mlstm = MomentumLSTMCell(input_size, hidden_size, mu, epsilon, bias, fg_init)
    self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

  def forward(self, x):
    # initial hidden state
    h0 = torch.zeros(x.size(0)*x.shape[2], self.hidden_size).to(device)
    c0 = torch.zeros(x.size(0)*x.shape[2], self.hidden_size).to(device)
    v = torch.zeros(x.shape[0]*x.shape[2], 4 * self.hidden_size).to(device)

    # Forward Prop
    out, _, _ = self.mlstm(x, (h0, c0), v)
    out = out.reshape(x.shape[0], -1)
    out = self.fc(out)
    return out

In [8]:
!pip install pytorch-forecasting
from pytorch_forecasting.metrics import MAPE

     |████████████████████████████████| 92kB 2.8MB/s 
     |████████████████████████████████| 266kB 9.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.8MB 11.9MB/s 
     |████████████████████████████████| 675kB 30.8MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
     |████████████████████████████████| 163kB 39.1MB/s 
     |████████████████████████████████| 276kB 41.5MB/s 
     |████████████████████████████████| 92kB 9.5MB/s 
     |████████████████████████████████| 133kB 28.9MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 112kB 35.6MB/s 
     |████████████████████████████████| 81kB 7.7MB/s 
  Created wheel for optuna: filename=optuna-2.3.0-cp36-none-any.whl size=359761 sha256=c3abc764db4864a485c915d42c7e99d10d33ac9c80521b1056e1b50849f11ef0
  Stored in directory: /root/.cache/pip/wheels/fa/91/19/64b0e

In [92]:
# function for saving a trained model
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
  print("=> Saving checkpoint")
  torch.save(state, filename)

# function for loading a saved model
def load_checkpoint(check_point):
  print("=> Loading checkpoint")
  model.load_state_dict(checkpoint['state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer'])

In [93]:
# Custom Dataset for Dataloader & Mini batch compatability
class MyDataset(Dataset):
    def __init__(self, data, window, target_cols):
        self.data = torch.Tensor(data)
        self.window = window
        self.target_cols = target_cols
        self.shape = self.__getshape__()
        self.size = self.__getsize__()
 
    def __getitem__(self, index):
        x = self.data[index:index+self.window]
        y = self.data[index+self.window,0:self.target_cols]
        return x, y
 
    def __len__(self):
        return len(self.data) -  self.window 
    
    def __getshape__(self):
        return (self.__len__(), *self.__getitem__(0)[0].shape)
    
    def __getsize__(self):
        return (self.__len__())

# function to load the m4 train/test data for a specified period length
def load_m4_data(time = "Daily", train=True):

  if train:
    url = 'https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Train/' + time + '-train.csv'
    filename = time.lower() + '_train.csv'
  else:
    url = 'https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Test/' + time + '-test.csv'
    filename = time.lower() + '_test.csv'

  res = requests.get(url, allow_redirects=True)

  with open(filename,'wb') as file:
      file.write(res.content)
  df = pd.read_csv(filename, index_col=0)

  return df

# function to compute loss on test set
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# function to make predictions for test set
def make_pred(model, lastTrain, predSize=14):
  with torch.no_grad():
    test_seq = lastTrain
    np.zeros(predSize)
    for i in range(predSize):
      y_test_pred = model(test_seq)
      pred = torch.flatten(y_test_pred).item()
      preds[i] = [pred]
      new_seq = test_seq.numpy().flatten()
      new_seq = np.append(new_seq, [pred])
      new_seq = new_seq[1:]
      test_seq = torch.as_tensor(new_seq).view(1, sequence_length, 1).float()
  return np.array(preds)

# prints the dimensions of a dataset
def print_dims(df, name):
  print("{} shape = {}".format(name, df.shape))

In [94]:
# load daily training data
df_train = load_m4_data("Daily", train=True)

# load daily testing data
df_test = load_m4_data("Daily", train=False)

# print dimensions
print_dims(df_train, "Daily_train")
print_dims(df_test, "Daily_test")

Daily_train shape = (4227, 9919)
Daily_test shape = (4227, 14)


In [95]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Hyperparameters
in_channel = 1 # CNN
# input_size = 784 # FCN
input_size = 1 # RNN
sequence_length = 28 # RNN
num_layers = 2
hidden_size = 256
num_classes = 1
learning_rate = 0.001
batch_size = 64
num_epochs = 2
load_model = False
# mu = 0.6 # MomentumLSTM
# epsilon = 0.6 # MomentumLSTM

cpu


In [96]:
# extract series
series = df_train.iloc[0, :]
series.dropna(inplace=True)

# Expand the dimension from [N,] => [N,1]
series = np.expand_dims(series, axis=1)
labels = np.expand_dims(df_test.iloc[0, :], axis=1)

# Fit a min max scaler to this series
scaler = MinMaxScaler()
scaler = scaler.fit(series)

# Transform the series and labels
train_data = scaler.transform(series)
test_data = scaler.transform(labels)

train_dataset = MyDataset(data=train_data, window=sequence_length, target_cols=1)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)

print(train_dataset.shape)
print(test_data.shape)

(978, 28, 1)
(14, 1)


In [97]:
# Initialize Network
# model = NN(input_size=input_size, num_classes=num_classes).to(device)
# model = CNN(in_channel=in_channel, num_classes=num_classes).to(device)
# model = RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes).to(device)
# model = GRU(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes).to(device)
model = LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes).to(device)
# model = BRNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes).to(device)
# model = MomentumLSTM(input_size=input_size, hidden_size=hidden_size, mu=mu, epsilon=epsilon).to(device)

# Loss and Optimizer
criterion = MAPE()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

if load_model:
  load_checkpoint(torch.load("my_checkpoint.pth.tar"))

In [99]:
def train_model(model, criterion, optimizer, train_loader, FCN=False):
  # Load trained model if one exists
  if load_model:
    load_checkpoint(torch.load("my_checkpoint.pth.tar"))

  # Train Network
  for epoch in range(10):
    losses = []

    if load_model:
      if epoch % 100 == 0:
        checkpoint = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
        save_checkpoint(checkpoint)

    for batch_idx, (data, targets) in enumerate(train_loader):
      if not FCN:
        data = data.to(device=device).squeeze(1) # ON FOR RNN/LSTM compatibility
      data = data.to(device=device)
      targets = targets.to(device=device)
      
      # flatten tensor to correct shape
      if FCN:
        data = data.reshape(data.shape[0], -1) # ON FOR FCN compatability
      
      # forward pass
      scores = model(data)
      loss = criterion(scores, targets)
      losses.append(loss.item())

      # backward
      optimizer.zero_grad()
      loss.backward()

      # gradient descent or adam step
      optimizer.step()

    mean_loss = sum(losses)/len(losses)
    if epoch % 50 == 0:
      print("Loss at epoch {} was {}".format(epoch, mean_loss))

In [84]:
# Create rolling window data
X_test, _ = sliding_windows(train_data[-29:], sequence_length)
print(X_test.shape)

# Create torch tensors
X_test = torch.from_numpy(X_test).float().to(device)

(1, 28, 1)


In [85]:
preds = make_pred(model, X_test)
print(mean_absolute_percentage_error(test_data, preds))

98.85925130954041


In [ ]:
# FCN Train
# Got 58991 / 60000 with accuracy 98.32
# FCN Test
# Got 9693 / 10000 with accuracy 96.93

# CNN Train
# Got 59397 / 60000 with accuracy 99.00
# CNN Test
# Got 9850 / 10000 with accuracy 98.50

# RNN Train
# Got 59076 / 60000 with accuracy 98.46
# RNN Test
# Got 9796 / 10000 with accuracy 97.96

# GRU Train
# Got 59691 / 60000 with accuracy 99.48
# GRU Test
# Got 9893 / 10000 with accuracy 98.93

# LSTM Train
# Got 59788 / 60000 with accuracy 99.65
# LSTM Test
# Got 9892 / 10000 with accuracy 98.92

# BRNN Train
# Got 59767 / 60000 with accuracy 99.61
# BRNN Test
# Got 9917 / 10000 with accuracy 99.17

# MomentumLSTM Train
# Got 58463 / 60000 with accuracy 97.44
# MomentumLSTM Test
# Got 9648 / 10000 with accuracy 96.48

Bring in data from M4 - daily data. The testing dataset is 15 days, so it looks like that will be the number of days forward we are trying to predict. 

In [ ]:
''' this loop will control the entire procedure but it is not done so I have commented it out. I am testing the same code on a single iteration below - Vai '''

# for row, series in df_train.iterrows():

#   # extract series
#   series = df_train.iloc[row, :]
#   series.dropna(inplace=True)

#   # Expand the dimension from [N,] => [N,1]
#   series = np.expand_dims(series, axis=1)
#   labels = np.expand_dims(df_test.iloc[row, :], axis=1)

#   # Fit a min max scaler to this series
#   scaler = MinMaxScaler()
#   scaler = scaler.fit(series)

#   # Transform train and test data with scaler
#   train_data = scaler.transform(series)
#   test_data = scaler.transform(labels)

#   # Set window size to num of predictions 
#   seq_length = len(labels)

#   # roll the data
#   X_train, y_train = sliding_windows(train_data, seq_length)
#   # X_test, y_test = sliding_windows(test_data, seq_length)

#   # make tensors
#   X_train = torch.from_numpy(X_train).float()
#   y_train = torch.from_numpy(y_train).float()

#   # X_test = torch.from_numpy(X_test).float()
#   # y_test = torch.from_numpy(y_test).float()

' this loop will control the entire procedure but it is not done so I have commented it out. I am testing the same code on a single iteration below - Vai '

In [41]:
# returns the data in a stacked rolling window format
def sliding_windows(data, seq_length):
  xs = []
  ys = []

  for i in range(0, len(data) - seq_length):
    x = data[i:(i+seq_length)]
    y = data[(i+seq_length)]
    xs.append(x)
    ys.append(y)

  return np.array(xs), np.array(ys)

In [ ]:
# extract series
series = df_train.iloc[0, :]
series.dropna(inplace=True)

# Expand the dimension from [N,] => [N,1]
series = np.expand_dims(series, axis=1)
labels = np.expand_dims(df_test.iloc[0, :], axis=1)

# Fit a min max scaler to this series
scaler = MinMaxScaler()
scaler = scaler.fit(series)

# Transform the series and labels
train_data = scaler.transform(series)
test_data = scaler.transform(labels)

# Set window size to num of predictions 
seq_length = 14

# Create rolling window data
X_train, y_train = sliding_windows(train_data, seq_length, 14)
y_train = y_train.squeeze()
print(X_train.shape)
print(y_train.shape)
# X_test, y_test = sliding_windows(test_data, seq_length)

# Create torch tensors
X_train = torch.from_numpy(X_train).float().to(device)
y_train = torch.from_numpy(y_train).float().to(device)

# X_test = torch.from_numpy(X_test).float()
# y_test = torch.from_numpy(y_test).float()

(964, 14, 1)
(964, 14)


In [ ]:
# Create an LSTM
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, seq_length, num_layers=2):
    super(LSTM, self).__init__()
    self.hidden_size = hidden_size
    self.seq_length = seq_length
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
    self.fc = nn.Linear(hidden_size, out_features=14)

  def forward(self, x):
    # initial hidden state
    h0 = torch.zeros(self.num_layers, self.seq_length, self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers, self.seq_length, self.hidden_size).to(device)

    # Forward Prop
    out, _ = self.lstm(x.view(x.shape[0], self.seq_length, -1), (h0, c0))
    out = out.view(self.seq_length, x.shape[0], self.hidden_size)[-1]
    out = self.fc(out)
    return out

In [ ]:
def train_model(model, train_data, train_labels, validation_data=None, test_labels=None, num_epochs=60):

  # init the loss function
  loss_fn = MAPE()

  # init the optimizer
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  # init empty arrays to store results
  train_hist = np.zeros(num_epochs)
  test_hist = np.zeros(num_epochs)

  # train
  for t in range(num_epochs):

    # make predictions
    y_pred = model(X_train)

    # calculate loss
    loss = loss_fn(y_pred.float(), y_train)

    # print loss for this epoch
    if t % 20 == 0:
      print(f'Epoch {t} train loss. {loss.item()}')

    # record loss
    train_hist[t] = loss.item()

    # reset optimizer 
    optimizer.zero_grad()

    # backprop
    loss.backward()

    # update model
    optimizer.step()

  return model.eval(), train_hist, test_hist

In [ ]:
# instantiate model 
model = LSTM(1, 512, seq_length=seq_length, num_layers=2).to(device)

# train model
model, train_hist, test_hist = train_model(model, train_data=X_train, train_labels=y_train, num_epochs=500)

Epoch 0 train loss. 1959.8482666015625
Epoch 20 train loss. 388.1073303222656
Epoch 40 train loss. 230.57875061035156
Epoch 60 train loss. 194.63063049316406
Epoch 80 train loss. 124.95750427246094
Epoch 100 train loss. 136.7980499267578
Epoch 120 train loss. 128.8214874267578
Epoch 140 train loss. 169.71240234375
Epoch 160 train loss. 106.40131378173828
Epoch 180 train loss. 94.447021484375
Epoch 200 train loss. 148.4458770751953
Epoch 220 train loss. 111.57532501220703
Epoch 240 train loss. 81.54461669921875
Epoch 260 train loss. 87.56201171875
Epoch 280 train loss. 65.10360717773438
Epoch 300 train loss. 90.55396270751953
Epoch 320 train loss. 59.15509796142578
Epoch 340 train loss. 45.910343170166016
Epoch 360 train loss. 67.6359634399414
Epoch 380 train loss. 80.42500305175781
Epoch 400 train loss. 87.63569641113281
Epoch 420 train loss. 90.84622955322266
Epoch 440 train loss. 54.31886291503906
Epoch 460 train loss. 53.87678909301758
Epoch 480 train loss. 45.49597930908203


In [ ]:
with torch.no_grad():
  X_test = y_train[-1:].unsqueeze(2)
  pred = model(X_test)
  pred = torch.flatten(pred)
  pred = pred.cpu().unsqueeze(1).numpy()
  score = mean_absolute_percentage_error(test_data, pred)
  print(score)

100.15099968425578


In [ ]:
print(test_data)

[[1.00730699]
 [1.00326892]
 [1.01942121]
 [1.02903567]
 [1.03067013]
 [1.0364388 ]
 [1.02153639]
 [1.02442073]
 [1.02999712]
 [1.03345832]
 [1.03441977]
 [1.03826555]
 [1.05018748]
 [1.04711085]]


In [ ]:
print(pred)

[[-0.00166464]
 [-0.00421416]
 [-0.00215543]
 [ 0.00960575]
 [-0.00517582]
 [-0.00782692]
 [-0.00060005]
 [-0.00514338]
 [ 0.01110361]
 [ 0.00467707]
 [ 0.00264906]
 [-0.00489027]
 [-0.00826365]
 [-0.01006651]]


In [ ]:
# NEED TO IMPLEMENT REVERSE TRANSFORM TO GET ACTUAL PREDICITONS
# EXAMPLE BELOW:
# true_cases = scaler.inverse_transform(
#     np.expand_dims(y_test.flatten().numpy(), axis=0)
# ).flatten()

# predicted_cases = scaler.inverse_transform(
#   np.expand_dims(preds, axis=0)
# ).flatten()

(1006, 1)

In [ ]:
def make_df_for_stats(row, series):
  """
  Performs summary statistical calculations on a dataframe

  Returns: a dictionary with the stats as the value and the stat as the key
  """

  # compute summary statistics
  stats = {}
  stats['n'] = len(series)
  stats['average'] = series.mean()
  stats['max'] = series.max()
  stats['min'] = series.min()
  stats['standard_deviation'] = series.std()

  # get the true labels from the test data
  true = df_test.loc[f'{row}'].values

  # generate profit predictions
  fb_prophet_pred = fb_prophet(series, len(true))

  # compute and store the mean absolute percentage
  stats['fb_prophet_mape'] = mean_absolute_percentage_error(true, fb_prophet_pred)

  # return summary statistics and MAPE score
  return stats